In [17]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [18]:
# Model
m = gp.Model("district")
m

<gurobi.Model Continuous instance district: 0 constrs, 0 vars, Parameter changes: Username=(user-defined)>

In [19]:
state = 'RI'
population = "districting_2020\\2020\\{}\\counties\\graph\\{}.population".format(state, state)
dimacs = "districting_2020\\2020\\{}\\counties\\graph\\{}.dimacs".format(state, state)

with open('Numberofdistricts.txt', 'r') as file:
    for line in file:
        parts = line.strip().split('\t')
        if parts[0] == state:
            num_districts = int(parts[1])


with open(population, "r") as file:
    # Skip the first line
    next(file)
    # Count the number of lines, each line represents a county
    num_counties = sum(1 for line in file)

num_districts


2

In [20]:
## Reading distances from file
filename = "districting_2020\\2020\\{}\\counties\\graph\\{}_distances.csv".format(state, state)
d = np.genfromtxt(filename, delimiter=',', skip_header=1)
d = d[:, 1:]

## Reading populations from file
with open(population, 'r') as file:
    # Read the total population from the first line
    total_population_line = next(file).strip()  # Removes newline character at the end
    total_population = int(total_population_line.split('=')[1].strip())
    
    # Initialize a dictionary to store the rest of the data
    p = {}
    
    # Iterate over the remaining lines
    for line in file:
        # Each line is expected to have two values, separated by a space
        key, value = line.strip().split()  # Split by whitespace
        p[int(key)] = int(value)  # Convert both to integers and store in the dictionary

## Reading neighbors from file
neighbors = set()
with open(dimacs, 'r') as f:
    lines = f.readlines()
    for line in lines:
        if line.startswith('e'):
            _, node1, node2 = line.split()
            node1 = int(node1)
            node2 = int(node2)
            if (node1, node2) not in neighbors and (node2, node1) not in neighbors:
                neighbors.add((node1, node2))

list(neighbors)

[(0, 1), (2, 4), (1, 2), (3, 4), (0, 3), (2, 3), (1, 3)]

In [27]:
adjacency_matrix_file = r"districting_2020\\2020\\{}\\counties\\graph\\{}_distances.csv".format(state, state)

# Set decision variable x for each node i whether it is in district j
x = {}
for i in range(num_counties):
    for j in range(num_districts):
        x[i, j] = m.addVar(vtype=gp.GRB.BINARY, name="x_{}_{}".format(i, j))

## Set decision variable y for each edge in neighbors, between nodes i and k, for each district j
y = {}
for i, k in neighbors:
    for j in range(num_districts):
        y[i, k, j] = m.addVar(vtype=gp.GRB.BINARY, name="y_{}_{}_{}".format(i, k, j))

# for i in range(num_counties):
#     for k in range(num_counties):
#         for j in range(num_districts):
#             y[i, k, j] = m.addVar(vtype=gp.GRB.BINARY, name="y_{}_{}_{}".format(i, k, j))

# Set slack variable
slack = {}
for j in range(num_districts):
    slack[j] = m.addVar(obj=1, name="slack")




alpha = 1  # Weight of the balanced weight term
y


{(0, 1, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 1, 1): <gurobi.Var *Awaiting Model Update*>,
 (2, 4, 0): <gurobi.Var *Awaiting Model Update*>,
 (2, 4, 1): <gurobi.Var *Awaiting Model Update*>,
 (1, 2, 0): <gurobi.Var *Awaiting Model Update*>,
 (1, 2, 1): <gurobi.Var *Awaiting Model Update*>,
 (3, 4, 0): <gurobi.Var *Awaiting Model Update*>,
 (3, 4, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 3, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 3, 1): <gurobi.Var *Awaiting Model Update*>,
 (2, 3, 0): <gurobi.Var *Awaiting Model Update*>,
 (2, 3, 1): <gurobi.Var *Awaiting Model Update*>,
 (1, 3, 0): <gurobi.Var *Awaiting Model Update*>,
 (1, 3, 1): <gurobi.Var *Awaiting Model Update*>}

In [28]:
# Set objective function
m.setObjective(gp.quicksum(y[i, k, j] * d[i, k] for i, k in neighbors for j in range(num_districts)) \
             + alpha * (gp.quicksum(slack[j] for j in range(num_districts))), gp.GRB.MINIMIZE)


In [30]:
## Add constraints

# Each edge in neighbors is assigned to exactly one district
for i, k in neighbors:
    for j in range(num_districts):
        m.addConstr(x[i, j] + x[k, j] <= 2 * y[i, k, j] )

# Add population constraint with slack variable
for j in range(num_districts):
    m.addConstr(gp.quicksum(x[i, j] * p[i] for i in range(num_counties)) >= ((total_population / num_districts) - slack[j]))

# Each county is assigned to exactly one district
for i in range(num_counties):
    m.addConstr(gp.quicksum(x[i, j] for j in range(num_districts)) == 1)



In [31]:
# Write LP formulation to a file
m.write("districts.lp")

# Read the LP formulation from the file
with open("districts.lp", "r") as f:
    lp_formulation = f.read()

# Print LP formulation
print("LP Formulation:")
print(lp_formulation)

# Solve
m.optimize()

LP Formulation:
\ Model district
\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  0 x_0_0 + 0 x_0_1 + 0 x_1_0 + 0 x_1_1 + 0 x_2_0 + 0 x_2_1 + 0 x_3_0
   + 0 x_3_1 + 0 x_4_0 + 0 x_4_1 + 0 y_0_1_0 + 0 y_0_1_1 + 0 y_2_4_0
   + 0 y_2_4_1 + 0 y_1_2_0 + 0 y_1_2_1 + 0 y_3_4_0 + 0 y_3_4_1 + 0 y_0_3_0
   + 0 y_0_3_1 + 0 y_2_3_0 + 0 y_2_3_1 + 0 y_1_3_0 + 0 y_1_3_1 + 0 slack
   + 0 slack + 0 x_0_0 + 0 x_0_1 + 0 x_1_0 + 0 x_1_1 + 0 x_2_0 + 0 x_2_1
   + 0 x_3_0 + 0 x_3_1 + 0 x_4_0 + 0 x_4_1 + 0 y_0_0_0 + 0 y_0_0_1
   + 0 y_0_1_0 + 0 y_0_1_1 + 0 y_0_2_0 + 0 y_0_2_1 + 0 y_0_3_0 + 0 y_0_3_1
   + 0 y_0_4_0 + 0 y_0_4_1 + 0 y_1_0_0 + 0 y_1_0_1 + 0 y_1_1_0 + 0 y_1_1_1
   + 0 y_1_2_0 + 0 y_1_2_1 + 0 y_1_3_0 + 0 y_1_3_1 + 0 y_1_4_0 + 0 y_1_4_1
   + 0 y_2_0_0 + 0 y_2_0_1 + 0 y_2_1_0 + 0 y_2_1_1 + 0 y_2_2_0 + 0 y_2_2_1
   + 0 y_2_3_0 + 0 y_2_3_1 + 0 y_2_4_0 + 0 y_2_4_1 + 0 y_3_0_0 + 0 y_3_0_1
   + 0 y_3_1_0 + 0 y_3_1_1 + 0 y_3_2_0 + 0 y_3_2_1 + 0 y_3_3_0 + 0 y_3_3_1
  

In [32]:
# Solve
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22000.2))

CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 21 rows, 166 columns and 64 nonzeros
Model fingerprint: 0x1d1c37b3
Variable types: 10 continuous, 156 integer (156 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+05]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+05]
Presolved: 15 rows, 22 columns, 50 nonzeros

Continuing optimization...


Cutting planes:
  Gomory: 1
  Cover: 6
  Clique: 3
  Relax-and-lift: 1

Explored 1 nodes (22 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 4: 112299 163108 248791 282727 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.122985000000e+05, best bound 1.122985000000e+05, gap 0.0000%


In [33]:
def printSolution():
    if m.status == GRB.OPTIMAL:
        print('\nObjective Value: %g' % m.ObjVal)
        for i in range(num_counties):
            for j in range(num_districts):
                print("x_{}_{} = {}".format(i, j, x[i, j].X))
        for j in range(num_districts):
            print("slack_{} = {}".format(j, slack[j].X))
    else:
        print('No solution')

printSolution()


Objective Value: 112298
x_0_0 = 1.0
x_0_1 = 0.0
x_1_0 = 0.0
x_1_1 = 1.0
x_2_0 = -0.0
x_2_1 = 1.0
x_3_0 = -0.0
x_3_1 = 1.0
x_4_0 = -0.0
x_4_1 = 1.0
slack_0 = 0.0
slack_1 = 112051.5
